# E-commerce Clickstream Data Analysis

This notebook provides sample analysis of the e-commerce clickstream data.

In [ ]:
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Database connection
DB_URL = f"postgresql://{os.getenv('POSTGRES_USER')}:{os.getenv('POSTGRES_PASSWORD')}@{os.getenv('POSTGRES_HOST')}:{os.getenv('POSTGRES_PORT')}/{os.getenv('POSTGRES_DB')}"
engine = create_engine(DB_URL)

print("Connected to database successfully!")

In [ ]:
# Query basic statistics
stats_query = """
SELECT 
    COUNT(*) as total_events,
    COUNT(DISTINCT user_id) as unique_users,
    COUNT(DISTINCT session_id) as total_sessions,
    COUNT(*) FILTER (WHERE event_type = 'page_view') as page_views,
    COUNT(*) FILTER (WHERE event_type = 'add_to_cart') as cart_adds,
    COUNT(*) FILTER (WHERE event_type = 'purchase') as purchases
FROM analytics.raw_events;
"""

stats_df = pd.read_sql(stats_query, engine)
print("Pipeline Statistics:")
print(stats_df.T)

In [ ]:
# Analyze event types distribution
event_dist_query = """
SELECT 
    event_type,
    COUNT(*) as count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as percentage
FROM analytics.raw_events 
GROUP BY event_type
ORDER BY count DESC;
"""

event_dist_df = pd.read_sql(event_dist_query, engine)

plt.figure(figsize=(10, 6))
plt.pie(event_dist_df['count'], labels=event_dist_df['event_type'], autopct='%1.1f%%')
plt.title('Distribution of Event Types')
plt.show()

print(event_dist_df)

In [ ]:
# Session analysis
session_query = """
SELECT 
    session_duration_minutes,
    page_views,
    add_to_cart_events,
    purchases,
    converted,
    total_purchase_amount
FROM analytics.user_sessions
WHERE session_duration_minutes IS NOT NULL;
"""

sessions_df = pd.read_sql(session_query, engine)

plt.figure(figsize=(15, 10))

# Session duration distribution
plt.subplot(2, 2, 1)
plt.hist(sessions_df['session_duration_minutes'], bins=30, alpha=0.7)
plt.title('Session Duration Distribution')
plt.xlabel('Duration (minutes)')
plt.ylabel('Frequency')

# Page views vs conversions
plt.subplot(2, 2, 2)
converted = sessions_df[sessions_df['converted'] == True]
not_converted = sessions_df[sessions_df['converted'] == False]

plt.hist([not_converted['page_views'], converted['page_views']], 
         bins=20, alpha=0.7, label=['Not Converted', 'Converted'])
plt.title('Page Views: Converted vs Not Converted')
plt.xlabel('Page Views')
plt.ylabel('Frequency')
plt.legend()

# Conversion rate by session length
plt.subplot(2, 2, 3)
sessions_df['duration_bucket'] = pd.cut(sessions_df['session_duration_minutes'], 
                                       bins=[0, 5, 10, 15, 30, float('inf')], 
                                       labels=['0-5min', '5-10min', '10-15min', '15-30min', '30min+'])
conversion_by_duration = sessions_df.groupby('duration_bucket')['converted'].mean()
conversion_by_duration.plot(kind='bar')
plt.title('Conversion Rate by Session Duration')
plt.ylabel('Conversion Rate')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

print("Session Analysis Summary:")
print(f"Average Session Duration: {sessions_df['session_duration_minutes'].mean():.2f} minutes")
print(f"Overall Conversion Rate: {sessions_df['converted'].mean()*100:.2f}%")
print(f"Average Page Views per Session: {sessions_df['page_views'].mean():.2f}")

In [ ]:
# Hourly traffic patterns
hourly_query = """
SELECT 
    hour_timestamp,
    total_events,
    unique_users,
    page_views,
    cart_additions,
    purchases,
    conversion_rate
FROM analytics.hourly_metrics
ORDER BY hour_timestamp;
"""

hourly_df = pd.read_sql(hourly_query, engine)
hourly_df['hour_timestamp'] = pd.to_datetime(hourly_df['hour_timestamp'])

if not hourly_df.empty:
    plt.figure(figsize=(15, 8))
    
    plt.subplot(2, 1, 1)
    plt.plot(hourly_df['hour_timestamp'], hourly_df['total_events'], marker='o')
    plt.title('Hourly Traffic Volume')
    plt.ylabel('Total Events')
    
    plt.subplot(2, 1, 2)
    plt.plot(hourly_df['hour_timestamp'], hourly_df['conversion_rate'], marker='o', color='red')
    plt.title('Hourly Conversion Rate')
    plt.ylabel('Conversion Rate (%)')
    plt.xlabel('Time')
    
    plt.tight_layout()
    plt.show()
else:
    print("No hourly metrics data available yet.")